### 1. Import needed packages and clone the dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from sklearn.datasets import load_files
from keras.utils import np_utils, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

print('You are using numpy version:', np.__version__)

!git clone https://github.com/Horea94/Fruit-Images-Dataset

train_dir = 'C:/Users/crysi/Private Dokumente/Papers & wissenschaftl. Arbeiten/Master/Foundations of Machine Learning/First Project/Pycharm/Fruit-Images-Dataset/Training/'
test_dir = 'C:/Users/crysi/Private Dokumente/Papers & wissenschaftl. Arbeiten/Master/Foundations of Machine Learning/First Project/Pycharm/Fruit-Images-Dataset/Test/'

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files, targets, target_labels

names_train, intclass_train, stringclass_train = load_dataset(train_dir)
names_test, intclass_test, stringclass_test = load_dataset(test_dir)

n_classes = len(np.unique(intclass_test))

train_hot_class = np_utils.to_categorical(intclass_train, n_classes)
test_hot_class = np_utils.to_categorical(intclass_test, n_classes)

def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

train_images_array = np.array(convert_image_to_array(names_train))
test_images_array = np.array(convert_image_to_array(names_test))

train_images_array = train_images_array.astype('float32')/255
test_images_array = test_images_array.astype('float32')/255

from sklearn.model_selection import train_test_split
#split training into another training and test (to reduce computation resources/time)
train_images_array, new_test_images_array, train_hot_class, new_test_hot_class = train_test_split(train_images_array, train_hot_class, test_size = 0.3, random_state = 42)

train_images_array, valid_images_array, train_hot_class, valid_hot_class = train_test_split(train_images_array, train_hot_class, test_size = 0.2)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    #model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    #model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    #model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    #model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.5))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.5))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.5))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
train_hot_class = np_utils.to_categorical(intclass_train, n_classes)
test_hot_class = np_utils.to_categorical(intclass_test, n_classes)

train_images_array = np.array(convert_image_to_array(names_train))
test_images_array = np.array(convert_image_to_array(names_test))

train_images_array = train_images_array.astype('float32')/255
test_images_array = test_images_array.astype('float32')/255

from sklearn.model_selection import train_test_split
#split training into another training and test (to reduce computation resources/time)
train_images_array, new_test_images_array, train_hot_class, new_test_hot_class = train_test_split(train_images_array, train_hot_class, test_size = 0.3, random_state = 42)

train_images_array, valid_images_array, train_hot_class, valid_hot_class = train_test_split(train_images_array, train_hot_class, test_size = 0.2)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    #model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    #model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    #model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    #model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.5))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.5))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.5))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(5))
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))

    #hiddenlayer 2
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(50)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.25))

    #hiddenlayer 4
    #model.add(Dense(300)) #900
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [ ]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann1'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

In [ ]:

history2=np.load('my_history.npy',allow_pickle='TRUE').item()


In [ ]:
plt.plot(history2['accuracy'])

In [ ]:
new_model = tf.keras.models.load_model('model_ann1.h5')
new_model.summary()

Model accuracy

In [ ]:
plt.figure(figsize = (6, 5))
plt.plot(history.history['accuracy'], color = 'r')
plt.plot(history.history['val_accuracy'], color = 'b')
plt.title('Model Accuracy', weight = 'bold', fontsize = 16)
plt.ylabel('accuracy', weight = 'bold', fontsize = 14)
plt.xlabel('epoch', weight = 'bold', fontsize = 14)
plt.ylim(0.6, 1.1)
plt.xticks(weight = 'bold', fontsize = 12)
plt.yticks(weight = 'bold', fontsize = 12)
plt.legend(['train', 'val'], loc = 'lower right', prop = {'size': 14})
plt.grid(color = 'y', linewidth = '0.5')
plt.show()

Show prediction.

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(1)  
   
 # summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc = 'lower right')
   
 # summarize history for loss  
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc = 'upper right')
plt.show()